### Passos necessários

1. Ler Embeddings criados,
2. Ler uma nova imagem, fazer pre-processamento e criar embeddings
3. Fazer a similaridades com as imagens da base

In [ ]:
# Lendo as bibliotecas
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
#import requests
#import os
import torch

#import io
from rembg import remove



import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

import faiss


In [ ]:
lista = pd.read_csv("database/indices_imagens_preprocessadas.csv")

In [ ]:
# Ler o índice do arquivo
index = faiss.read_index('faiss_index_embeddings/vector_databases_faiss_index.index')  # Carrega o índice de um arquivo


In [ ]:
print(f"Número de itens no índice: {index.ntotal}")

In [ ]:
# Lendo algoritmo ViT - PreTrained
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTModel.from_pretrained('google/vit-base-patch16-224')

In [ ]:
# Processar a nova imagem
arquivo = "database/natural_images/airplane/airplane_0004.jpg"
#arquivo = "Imagens_teste/gato3.jpeg"
imagem = Image.open(arquivo)
img_redimensionada = imagem.resize((224, 224), Image.LANCZOS) # Usando LANCZOS para melhor qualidade
bytes_saida = remove(img_redimensionada)
novaimagem = bytes_saida.convert('RGB')

In [ ]:
largura_celula = 224
altura_celula = 224

largura_total = largura_celula * 2
altura_total = altura_celula
grade_final = Image.new('RGB', (largura_total, altura_total))

grade_final.paste(img_redimensionada, (0, 0))
grade_final.paste(novaimagem, (largura_celula, 0))


In [ ]:
grade_final

In [ ]:
inputs = processor(images=novaimagem, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [1, 768]
nova_embedding = cls_embedding.numpy()
# Normalize os embeddings (para usar distância do cosseno)
nova_embedding = nova_embedding / np.linalg.norm(nova_embedding, axis=1, keepdims=True)

In [ ]:
# Buscando as k imagens mais próximas no índice
k = 4  # Número de imagens semelhantes a buscar
distances, indices = index.search(nova_embedding, k)
# Imagens Retornadas
imagens_retornadas = [i for i in indices[0]]
print(imagens_retornadas)

In [ ]:
# Valores da distância do Cosseno
distances

In [ ]:
# Criando uma sublista apenas com as imagens retornadas
sublista = lista[lista['indice'].isin(imagens_retornadas)]

In [ ]:
# Criando uma array com das imagens
novasimagem = []

for i in range(len(sublista)):
    imgpath = sublista.iloc[i][1]
    # Processar a nova imagem
    imagem = Image.open(imgpath)
    img_redimensionada = imagem.resize((640, 480), Image.LANCZOS) # Usando LANCZOS para melhor qualidade
    novaimagem = img_redimensionada.convert('RGB')
    novasimagem.append(novaimagem)

In [ ]:
largura_celula = 640
altura_celula = 480

largura_total = largura_celula * 2
altura_total = altura_celula * 2
grade_final = Image.new('RGB', (largura_total, altura_total))

grade_final.paste(novasimagem[0], (0, 0))
grade_final.paste(novasimagem[1], (largura_celula, 0))
grade_final.paste(novasimagem[2], (0, altura_celula))
grade_final.paste(novasimagem[3], (largura_celula, altura_celula))

# Mostra a imagem da grade na tela
print("Exibindo a grade de imagens na tela...")
grade_final.show()